In [135]:
import pandas as pd
import requests
import json
from collections import OrderedDict
from pymongo import MongoClient
import http.client, urllib.parse
import ipynb.fs.full.config as cfg

def get_news(articles=100):
    #Connect to API
    conn = http.client.HTTPConnection('api.mediastack.com')
    #Set parameters
    params = urllib.parse.urlencode({
        'access_key': cfg.creds['access_key'],
        'keywords': 'cryptocurrency', 'bitcoin'
        'sort': 'published_desc',
        'countries': 'us',
        'limit': articles,
        })
    #Request Data
    conn.request('GET', '/v1/news?{}'.format(params))
    #Parse Request
    res = conn.getresponse()
    result = json.loads(res.read())
    data = pd.DataFrame(columns=['author','title','description','url','source','image','category','language','country','published_at'])
    for i in result['data']:
        data = data.append(i,ignore_index=True)
    data.drop(['image','category', 'language', 'country'], axis=1)
    data['published_at'] = pd.to_datetime(data['published_at'], format='%Y-%m-%d').dt.date
    data['published_at'] = pd.to_datetime(data['published_at'], format='%Y-%m-%d %H:%M:%S.')
    if data is None:
        print("Did not return any data")
    else:
        od = data.to_dict('records', into=OrderedDict())
    return od